In [7]:
import mysql.connector
import pickle
from jieba import analyse
import pickle
import os
import jieba
from jieba import analyse
from snownlp import SnowNLP
from snownlp.sentiment import Sentiment
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt

In [8]:
#输入：微博名
#输出： information：用户信息, senti：积极消极数, big5：大五人格得分以及评价,keywords：top50得词频
###information中各字段：'U_ID'：自动生成的ID； U_user_id', 用户ID；'U_name'真实姓名；'U_nick_name'用户昵称；'U_location'用户所在地；
#'U_sex'用户性别；'U_sexual_orientation'用户性取向；'U_Emotional_status'情感状态；'U_Birthday'生日；'U_constellation'星座；
#'U_Blood'血型；'U_own_bolg_address_name'个人微博地址；'U_brief_introduction'个人简介；'U_Registration_time'注册时间；'U_QQ'QQ
#'U_email'email;'U_MSN'MSN;'U_Work',工作信息;'U_educational'教育信息;'U_fans_Number'粉丝数;'U_blog_Number'博文数;"U_concerns_Number"关注数
#'U_credit'信誉度；
#'U_Head_Photo'数据库中的该字段用于存放图片的路径 是空的

def weibo_user(name):
    # 连接数据库
    coon = mysql.connector.connect(user='weibo_user', password='ir@DUT#weibo', host="10.8.128.205",
                                   port=23306)
    cursor = coon.cursor()
    cursor.execute('use weiboDB;')
    sql_str = 'select * from user_info_table3 where U_nick_name = "{0}";'.format(name)
    cursor.execute(sql_str)
    
    res = cursor.fetchall()
    for i in res:
        information = {"U_user_id":i[1],"U_name":i[2],"U_nick_name":i[3],"U_location":i[4],"U_sex":i[5],"U_sexual_orientation":i[6],
                 "U_Emotional_status":i[7],"U_Birthday":i[8],"U_constellation":i[9],"U_Blood":i[11],"U_own_bolg_address_name":i[12],
                 "U_brief_introduction":i[13],"U_Registration_time":i[14],"U_QQ":i[15],"U_email":i[16],"U_MSN":i[17],"U_Work":i[18],
                 "U_educational":i[19],"U_fans_Number":i[20],"U_concerns_Number":i[27],"U_blog_Number":i[21],"U_credit":i[22]}
        senti = {"U_senti":i[24]}
        
        big = i[25][1:-1].split(",")
        dict_big5 = {'责任性':int(big[0]),'外倾性':big[1],'开放性':big[2],'神经质性':big[3],'宜人性':big[4]}
        big5 = {"U_big5":dict_big5,"U_big5_comment":i[26]}
    sql_str1 =   'select W_content from weibo_table where W_nick_name = "{0}";'.format(name)
    cursor.execute(sql_str1)
    res = cursor.fetchall()
    tfidf = analyse.extract_tags
    key_word =[]
    for i in res: 
        keywords = tfidf(i[0],allowPOS=('ns','nr','nt','nz','nl','n', 'vn','vd','vg','v','vf','a','an','i'))
        result=[]
        for j in keywords:    
            result.append(j)
        key_word.append(" ".join(result)) 
    words = ""
    for line in key_word:
        for i in line:
            words = words+i
    result=jieba.analyse.textrank(words,topK=50,withWeight=True)
    keywords = dict()
    for i in result:
        keywords[i[0]]=i[1]
    cursor = coon.close()
    return information, senti, big5,keywords

In [9]:
information, senti, big5,keywords = weibo_user("谢磊NPU")

In [33]:
keywords = [{'name':str(k),'value':float(keywords[k])} for k in keywords]
senti_value = [int(i.strip()) for i in senti['U_senti'][1:-1].split(',')]
senti_result = [{'name':'pos','value':float(senti_value[0])},{'name':'neg','value':float(senti_value[1])}]
big5['U_big5'] = [{'name':str(k),'value':float(big5['U_big5'][k])} for k in big5['U_big5']]

In [36]:
result = {'infoDetail':information,
'sentiPie':senti_result,
 'big5':big5,
'kwordCloud':keywords,}

In [37]:
result

{'infoDetail': {'U_user_id': '2627058773',
  'U_name': ' ',
  'U_nick_name': '谢磊NPU',
  'U_location': '陕西 西安',
  'U_sex': '男',
  'U_sexual_orientation': ' ',
  'U_Emotional_status': ' ',
  'U_Birthday': ' ',
  'U_constellation': ' ',
  'U_Blood': ' ',
  'U_own_bolg_address_name': 'https://weibo.com/xielei21st?from=inf&wvr=5&loc=infdomain',
  'U_brief_introduction': '西北工业大学计算机学院教授，从事智能语音处理、机器学习相关研究。研究兴趣：语音识别、语音合成、口语内容分析、音视频多模态处理等。',
  'U_Registration_time': datetime.datetime(2012, 2, 22, 0, 0),
  'U_QQ': ' ',
  'U_email': ' ',
  'U_MSN': ' ',
  'U_Work': ' ',
  'U_educational': '  学位：大学  学校： 西北工业大学 时间：2011年',
  'U_fans_Number': 3696,
  'U_concerns_Number': 412,
  'U_blog_Number': 9016,
  'U_credit': '信用较好'},
 'sentiPie': [{'name': 'pos', 'value': 230}, {'name': 'neg', 'value': 11}],
 'big5': {'U_big5': [{'name': '责任性', 'value': 13},
   {'name': '外倾性', 'value': ' 5'},
   {'name': '开放性', 'value': ' 17'},
   {'name': '神经质性', 'value': ' 13'},
   {'name': '宜人性', 'value': ' 21'}],
  'U_big5_c